In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import shuffle
from sklearn.model_selection import LeaveOneOut
import itertools

file_name = 'Dados_Forchheimer.xlsx'
file_path = os.path.join(os.getcwd(), file_name)

ANNDF = pd.read_excel(file_path, sheet_name='Train')

X = ANNDF.iloc[:, 0:3 ].values
Y = ANNDF.iloc[:, 3].values.reshape(-1, 1)

X, Y = shuffle(X, Y)

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, train_size=0.75)  # 25% for testing and shuffle

scalerx = StandardScaler()
scalery = StandardScaler()

Xtrain = scalerx.fit_transform(xtrain)
Ytrain = scalery.fit_transform(ytrain)

loocv = LeaveOneOut()
loocv.get_n_splits(X)

mlp = MLPRegressor(max_iter=5000)

nneurons = [32, 64, 128, 256]
parameters = {
    'hidden_layer_sizes': [],
    'activation': ['relu', 'logistic'],
    'solver': ['adam']
}

# Gerar configurações de 1 a 4 camadas
for n_layers in range(2, 6):  # 1, 2, 3 e 4 camadas
    for hlayers in itertools.product(nneurons, repeat=n_layers):
        parameters['hidden_layer_sizes'].append(hlayers)

ypred_or = []
ytest_val = []
bestparam = []
bestac    = []

for i, (train_index, test_index) in enumerate(loocv.split(X)):
    print(i+1)
    xtrain = X[train_index,:]
    xtest  = X[test_index,:]

    ytrain = Y[train_index,:]
    ytest  = Y[test_index,:]

    scalerx = StandardScaler()
    scalery = StandardScaler()

    Xtrain = scalerx.fit_transform(xtrain)
    Ytrain = scalery.fit_transform(ytrain)

    kf = KFold(n_splits=5, shuffle=True) #max 5...

    gsearchcv = GridSearchCV(estimator=mlp, param_grid=parameters, cv=kf, verbose=0)
    gsearchcv.fit(Xtrain, Ytrain.ravel())

    bestac.append(gsearchcv.best_score_ * 100)  # Convert to percentage
    bestparam.append(gsearchcv.best_params_)

    print('Best Accuracy: {:.2f} %'.format(bestac[i]))
    print('Best Parameters:', bestparam[i])

    Xtest_scaled = scalerx.transform(xtest)
    ypred = gsearchcv.predict(Xtest_scaled).reshape(-1, 1)
    ypred_or.append(scalery.inverse_transform(ypred))
    ytest_val.append(ytest)

import joblib

# Após o treinamento do modelo
joblib.dump(gsearchcv.best_estimator_, 'mlp_model_forch_loocv.pkl')
joblib.dump(scalerx, 'scalerx_mlp_model_forch_loocv.pkl')
joblib.dump(scalery, 'scalery_mlp_model_forch_loocv.pkl')

1
Best Accuracy: 99.86 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (256, 256, 128, 256, 64), 'solver': 'adam'}
2
Best Accuracy: 99.87 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (128, 256, 128, 256, 256), 'solver': 'adam'}
3
Best Accuracy: 99.83 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (128, 128, 256, 256, 128), 'solver': 'adam'}
4
Best Accuracy: 99.88 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (128, 256, 256, 256, 256), 'solver': 'adam'}
5
Best Accuracy: 99.87 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (64, 128, 256, 256), 'solver': 'adam'}
6
Best Accuracy: 99.75 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (128, 128, 256, 256, 128), 'solver': 'adam'}
7
Best Accuracy: 99.83 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (64, 128, 256, 256, 64), 'solver': 'adam'}
8
Best Accuracy: 99.83 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes'

['scalery_mlp_model_forch.pkl']

In [1]:
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_percentage_error

ypred_or = [items for subList in ypred_or for items in subList]
ytest_val = [items for subList in ytest_val for items in subList]

r2 = r2_score(ytest_val, ypred_or)
rmse = root_mean_squared_error(ytest_val, ypred_or)
mape = mean_absolute_percentage_error(ytest_val, ypred_or)

NameError: name 'ypred_or' is not defined

In [14]:
import joblib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

mlp_model = joblib.load('mlp_model_forch_loocv.pkl')
scalerx = joblib.load('scalerx_mlp_model_forch_loocv.pkl')
scalery = joblib.load('scalery_mlp_model_forch_loocv.pkl')

import numpy as np

novo_valor = np.array([[0.426272258064516, 1, 12]])  # Note que o formato deve ser (1, n_features)

novo_valor_scaled = scalerx.transform(novo_valor)

pred_scaled = mlp_model.predict(novo_valor_scaled)

pred = scalery.inverse_transform(pred_scaled.reshape(-1, 1))

print("Predição:", pred)

Predição: [[89640.12640573]]


In [3]:
import joblib
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
import pandas as pd
import os

# Carregar o modelo e os scalers
mlp_model = joblib.load('mlp_model_forch_loocv.pkl')
scalerx = joblib.load('scalerx_mlp_model_forch_loocv.pkl')
scalery = joblib.load('scalery_mlp_model_forch_loocv.pkl')

# Carregar os dados
file_name = 'Dados_Forchheimer.xlsx'
file_path = os.path.join(os.getcwd(), file_name)

ANNDF = pd.read_excel(file_path, sheet_name='Train')

X = ANNDF.iloc[:, 0:3].values
Y = ANNDF.iloc[:, 3].values.reshape(-1, 1)

# Escalar os dados
X_scaled = scalerx.transform(X)
Y_scaled = scalery.transform(Y)

# Fazer predição
Y_pred_scaled = mlp_model.predict(X_scaled).reshape(-1, 1)

# Desfazer o scaling
Y_pred = scalery.inverse_transform(Y_pred_scaled)
Y_true = Y  # Já estão no valor real

# Calcular métricas
r2 = r2_score(Y_true, Y_pred)
rmse = np.sqrt(mean_squared_error(Y_true, Y_pred))
mape = mean_absolute_percentage_error(Y_true, Y_pred) * 100  # Em %

# Informações da rede
hidden_layers = mlp_model.hidden_layer_sizes
if isinstance(hidden_layers, int):
    hidden_layers = (hidden_layers,)  # Corrigir para tupla se for só uma camada

print("======== INFORMAÇÕES DO MODELO ========")
print(f"Número de camadas ocultas: {len(hidden_layers)}")
print(f"Número de neurônios por camada: {hidden_layers}")
print(f"R²: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f} %")

# Predição para novo valor
novo_valor = np.array([[0.426272258064516, 1, 12]])  # (1, n_features)

novo_valor_scaled = scalerx.transform(novo_valor)
pred_scaled = mlp_model.predict(novo_valor_scaled)
pred = scalery.inverse_transform(pred_scaled.reshape(-1, 1))

print(f"Predição para o novo ponto {novo_valor.flatten()}: {pred.flatten()[0]:.4f}")


======== INFORMAÇÕES DO MODELO ========
Número de camadas ocultas: 5
Número de neurônios por camada: (64, 64, 128, 256, 64)
R²: 0.9988
RMSE: 12924.6599
MAPE: 20.33 %
Predição para o novo ponto [ 0.42627226  1.         12.        ]: 89640.1264


In [5]:
import joblib
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Carregar modelo e scalers
mlp_model = joblib.load('mlp_model_forch_loocv.pkl')
scalerx = joblib.load('scalerx_mlp_model_forch_loocv.pkl')
scalery = joblib.load('scalery_mlp_model_forch_loocv.pkl')

# Criar malha de entrada (vazão x espessura)
vazao_vals = np.linspace(0, 1, 50)
espessura_vals = np.linspace(1.0, 1.5, 50)
cell_size = 10.0  # fixo

V, E = np.meshgrid(vazao_vals, espessura_vals)

# Preparar a matriz de entradas [vazão, espessura, cell size]
X_input = np.column_stack((V.ravel(), E.ravel(), np.full(V.size, cell_size)))

# Escalar os dados
X_scaled = scalerx.transform(X_input)

# Prever valores de dP
Y_scaled = mlp_model.predict(X_scaled).reshape(-1, 1)
Y_pred = scalery.inverse_transform(Y_scaled).reshape(V.shape)

# Criar gráfico de superfície com Plotly
fig = go.Figure(data=[go.Surface(
    z=Y_pred,
    x=V,
    y=E,
    colorscale='Viridis',
    colorbar=dict(title='ΔP'),
)])

fig.update_layout(
    title='Superfície: ΔP vs Vazão e Espessura (cell size = 10)',
    scene=dict(
        xaxis_title='Vazão',
        yaxis_title='Espessura',
        zaxis_title='ΔP (dP)',
    ),
    width=900,
    height=700,
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()
